# Configuring the precision

Many scientists choose the double precision type (`double`) by default, out of habit. They suspect that simple precision (`float`) might be insufficient, and higher precision (`long double`) might be slow and bulky. Do not rely on such fuzzy intuition: instead, write your code with a configurable precision, and **try out** different ones.

### First pre-requisite : accuracy control

Before trying different floating-point types, one must wonder how to **validate how accurate is the final result**, or at least if it accurate enough.

### Second-prequisite : execution time control

When monitoring the execution time, there are a few rules to known about, especially when playing with small/toy code:
* run your program many times and compute a mean exwcution time ;
* your program must run long enough, so that the processor pipelines get filled and you go well beyond the initial *computing latency* ;
* if the size of your data become too big, you may fill out the processor cache memory, become I/O bound, and the results will not any more express the CPU performance, but the bandwidth of the memory bus.

## First step with `using`

If you are starting from a code written with `double`, you can simply search and replace all `double` with, for example, `real`, and add `using real = double` at the beginning of all your `*.cc` body files, or at the beginning of some `*.h` header file included everywhere. 

From there on, you can change your alias into `using real = float`, and recompile everything. Then check if the results are still accurate enough, and measure how faster is execution.

In [5]:
%%file tmp.precision.h

using real = double ;

Writing tmp.precision.h


In [7]:
%%file tmp.precision.cpp

#include "tmp.precision.h"
#include <iostream>
#include <vector>

void reduce( std::vector<real> const & collection )
 {
  real res {static_cast<real>(1.)} ;
  for ( auto element : collection )
   { res *= element ; }
  std::cout.precision(18) ;
  std::cout << res << std::endl ;
 }

int main()
 {
  reduce({ 1.1, 2.2, 3.3, 4.4, 5.5 }) ;
 }

Overwriting tmp.precision.cpp


In [8]:
!rm -f tmp.precision.exe && g++ -std=c++17 tmp.precision.cpp -o tmp.precision.exe

In [9]:
!./tmp.precision.exe

193.261200000000031


In [10]:
%%file tmp.precision.h

using real = float ;

Overwriting tmp.precision.h


In [11]:
!rm -f tmp.precision.exe && g++ -std=c++17 tmp.precision.cpp -o tmp.precision.exe

In [12]:
!./tmp.precision.exe

193.261199951171875


This approach requires a complete recompilation and works in an "all or nothing" mode. We can improve it a bit by using a collection of `real1`,` real2`,... type aliases, to be used in different sub-sections of the code.

BEWARE: in order not to stumble on unattended side effects, it is adviced to **track and remove from the original code any form of implicit conversion** between floating-point types.

## Go further with templates

For each portion of your code that can be configured separately, you can create a template, taking as a parameter the floating-point type to be used.

This flexibility comes with a price: the function bodies must be completely moved into header, which causes the usual lengthening of compilation and bloating of executables... until C++20 provide more efficient solutions using *Modules*.

Make all and every class a template will often imply adaptations:
* some nested types may become "dependent", and therefore require an additional `typename` keyword ;
* some inherited members may become unreachable when the base class become a template, and therefore require the addition of `this->` or `using` ;
* some implicit conversion may become dependent on a template parameter, not any more inferable by the compiler, and therefore require to be made explicit. 

In [17]:
%%file tmp.precision.cpp

#include <iostream>
#include <vector>

template< typename Real >
void reduce( std::vector<Real> const & collection )
 {
  Real res = 1. ;
  for ( auto element : collection )
   { res *= element ; }
  std::cout.precision(18) ;
  std::cout << res << std::endl ;
 }

int main()
 {
  reduce(std::vector<double>{ 1.1, 2.2, 3.3, 4.4, 5.5 }) ;
  reduce(std::vector<float>{ 1.1, 2.2, 3.3, 4.4, 5.5 }) ;
 }

Overwriting tmp.precision.cpp


In [18]:
!rm -f tmp.precision.exe && g++ -std=c++17 tmp.precision.cpp -o tmp.precision.exe

In [19]:
!./tmp.precision.exe

193.261200000000031
193.261199951171875


# Questions?

# Exercise

## Initial code

In the example given below we do a "SAXPY" calculation (`y = a*x+y`)
on a vector of elements `XY`. Understand this code and run it.

In [45]:
%%file tmp.precision.cpp

#include <iostream>
#include <cassert> // for assert
#include <cstdlib> // for rand
#include <valarray>

void randomize( std::valarray<double> & collection )
 {
  srand(1) ;
  for ( auto & elem : collection )
   { elem = std::rand()/(RAND_MAX+1.)-0.5 ; }
 }

double mean( std::valarray<double> & collection )
 {
  double res {0.} ;
  for ( auto elem : collection )
   { res += elem ; }
  return res/collection.size() ;
 }

int main( int argc, char * argv[] )
 {
  assert(argc==3) ;
  int size {atoi(argv[1])} ;
  int repeat {atoi(argv[2])} ;

  std::valarray<double> xs(0.,size) ;
  std::valarray<double> ys(0.,size) ;
  randomize(xs) ;
  while (repeat--)
    { ys = 0.1*xs + ys ; }
  double res = mean(ys) ;

  std::cout.precision(18) ;
  std::cout<<res<<std::endl ;
 }

Overwriting tmp.precision.cpp


The main program receives two arguments:
* the first argument sets up the size of the collections, and will affect the occupied RAM as well as the I/O volume ;
* the second argument sets up the number of times we repeat the main loop, and will affect the volume of calculations performed.

By fiddling with these two arguments we can modify the balance between calculation and data volume. Also, with low values, we can have a reference output, that we can quickly check when refactor the code. With high values, we can profile the performances.

In [48]:
%%file tmp.precision.sh
echo

opt=${1}
shift

rm -f tmp.precision.exe
g++ -O${opt} -std=c++17 tmp.precision.cpp -o tmp.precision.exe

./tmp.precision.exe $*

rm -f tmp.precision.py
echo "s = 0" >> tmp.precision.py
for i in 0 1 2 3 4 5 6 7 8 9 ; do
    \time -f "s += %U" -a -o ./tmp.precision.py ./tmp.precision.exe $*  >> /dev/null
done
echo "print(\"{:.4} s\".format(s/10.))" >> tmp.precision.py
python3 tmp.precision.py

echo

Overwriting tmp.precision.sh


The script receives three arguments:
* the first one sets the level of optimization requested from the compiler,
* the next two are passed as is to the executable.

In [50]:
! bash -l tmp.precision.sh 2 1024 100000


67.5053500207703507
0.109 s



## To do

Modify the code so that it can turn either in `float`, `double` or `long double`. Of course you can just recompile the code replacing all types each time. But rather try instead to template the code to the float type used and add an additional command line option in order to choose precision at execution.

Fill in the table below, which summarizes the results and computation times for different accuracies with the arguments `1024 100000`. Calculate the speedup as the ratio between the reference time for `double`/`-O2` and the current computation time. Determine the number of significant digits in the result by comparing with the reference result for `long double`.

| Type   | time (s) | speedup |              result | significant digits |
| :------| -------: | ------: | ------------------: | -----------------: |
| float  |          |         |                     |                    |
| double |          |     1.0 |                     |                    |
| long   |          |         |                     |                 18 |

Try the same with `g++ -O0`, `g++ -O1`, `g++ -O3`, `g++ -Ofast`...

© *CNRS 2021*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*